In [4]:
!pip install polyglot
!pip install PyICU
!pip install pycld2
!pip install Morfessor
import numpy as np
import nltk
from nltk.util import ngrams
from polyglot.text import Word
from polyglot.text import Text

     |████████████████████████████████| 133kB 8.2MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52558 sha256=bbb1919f8e3a782845cc481780c5c7ce781b2c4833ba0f8f84ff4320852272bb
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 10.3MB/s 
  Created wheel for PyICU: filename=PyICU-2.6-cp36-cp36m-linux_x86_64.whl size=1288440 sha256=b22e9948b459196dd7857d7ddbab2f92c7798943e588d478d237908d2eb7bfe0
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built PyICU
     |████████████████████████████████| 41.4MB 81kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833514 sha256=1f32ab20002cded987f4caed066b10139ca98dc83cf1981361c6b6ca41f2b33f
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3d

In [5]:
!polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


In [230]:
def external_features(headline, body):

  train_headline_sentences = []
  train_body_sentences = []
  train_headline_sentences.append(headline)
  train_body_sentences.append(body)

  ext = []
  i = 0
  for sent1,sent2 in zip(train_headline_sentences,train_body_sentences):
    print(i)
    i+=1
    vec = []
    #character ngrams
    for n in range(2,17):
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_2 = ngrams(sent2.lower(),n)
      vec.append(len(list(set(n_grams_1).intersection(n_grams_2))))

    #word ngrams
    for n in range(2,7):
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_2 = ngrams(sent2.lower().split(),n)
      vec.append(len(list(set(n_grams_1).intersection(n_grams_2))))

    #Sentence polarity
    flag=False
    text1 = Text(sent1)
    text2 = Text(sent2)
    pol1 = 0
    pol2 = 0
    for word in text1.words:
      try:
        pol1+=word.polarity
      except:
        flag=True
        vec.append(0)
        break
    if (flag==True):
      ext.append(vec)
      continue
    flag=False
    for word in text2.words:
      try:
        pol2+=word.polarity
      except:
        flag=True
        vec.append(0)
        break
    if (flag==True):
      ext.append(vec)
      continue
    
    
    pol1 = pol1/(len(sent1.split())*1.0)
    pol2 = pol2/(len(sent2.split())*1.0)
    print("pol1 : ", pol1)
    print("pol2 : ", pol2)
    print(pol1-pol2)
    vec.append(pol1-pol2)

    ext.append(vec)

  return np.array(ext)



In [184]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [244]:
def statistical_features(h, b):

  v1 = TfidfVectorizer(max_features= 2261)
  v2 = TfidfVectorizer(max_features= (10000- 2261))
  headline = []
  body = []
  headline.append(h)
  body.append(b)
  statistical_h = v1.fit_transform(headline)
  statistical_b = v1.fit_transform(body)
  #print(statistical_h.shape)
  a = statistical_h.toarray()
  #a.shape[1]
  b = np.zeros((1, 2261 - a.shape[1]))  
  #b.shape
  statistical_h = np.append(a, b).reshape((1, 2261))
  print(statistical_h.shape)
  print(statistical_b.shape)
  c = statistical_b.toarray()
  #c.shape[1]
  d = np.zeros((1, 10000 - 2261 - c.shape[1]))
  #d.shape
  statistical_b = np.append(c, d).reshape((1, 10000-2261))
  #print(statistical_b.shape)
  final_statistical_features = np.concatenate((statistical_h, statistical_b),axis = 1)
  print(final_statistical_features.shape)

  return final_statistical_features